# Example of packaging fits files

In [3]:
from copy import copy
from glob import glob

import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy.table import Table
from astropy.time import Time
from astropy.time import Time, TimeDelta

t0 = Time("2000-01-01T12:00:00.000", format="isot")
tpd = Time("2023-07-25T12:00:00.000", format="isot")


def fits_from_table(table, header=None):
    """Helper function to convert astropy.Table to astropy.fits.TableHDU"""
    cols = [
        fits.Column(col.name, col.dtype, unit=col.unit.name if col.unit is not None else None, array=col.data)
        for col in table.itercols()
    ]
    return fits.TableHDU.from_columns(cols, nrows=len(table), header=header)

In [23]:
# This is an astropy object that lets us get ra/dec etc
c = SkyCoord.from_name("GJ 1132")

# This is just random data as a place holder
dat = np.random.normal(0, 1, size=(3, 256, 256)).astype(np.int32)

# Fake ROI star parameters data
tab = Table(
    np.random.normal(0, 1, size=(8, 2)),
    names=["Column", "Row"],
    units=[u.pix, u.pix],
)


The primary extension of the fits file frequently doesn't have data, and is just a header. The primary header needs cards, we'll put in things that are about the observation as a whole. Some of these are standard headers, e.g. the top 5 are standard. Ideally, we would add comments to every one the cards. There are character limits on the lengths of keywords and card comments

In [31]:
cards = [
    ("SIMPLE", True, "conforms to FITS standard"),
    ("BITPIX", 16, "array data type"),
    ("NAXIS", dat.ndim, "number of dimensions"),
    ("EXTEND", True, "file contains extensions"),
    ("EXTNAME", "PRIMARY", "name of extension"),
    ("NEXTEND", 3, "number of standard extensions"),
    ("SIMDATA", False, "simulated data"),
    ("SCIDATA", True, "science data"),
    ("TELESCOP", "NASA Pandora", "telescope"),
    ("INSTRMNT", "VISDA", "instrument"),
    ("CREATOR", "Pandora MOC", "creator of this product"),
    ("CRSOFTV", "v0.1.0", "creator software version"),
    ("TARG_RA", c.ra.value, "target right ascension [deg]"),
    ("TARG_DEC", c.dec.value, "target declination [deg]"),
    ("FRMSREQD", 40, "number of frames requested"),
    ("FRMSCLCT", 40, "number of frames collected"),
    ("NUMCOAD", 1, "number of frames coadded"),
    ("EXPTIME", 0.2, "exposure time [s]"),
    ("EXPDELAY", 32, "exposure time [ms]"),
    ("RICEX", 5, "bit noise parameter for Rice compression"),
    ("RICEY", 10, "bit noise parameter for Rice compression"),
    ("CORSTIME", 744941360, "seconds since the TAI Epoch"),
    ("FINETIME", 792000000, "nanoseconds added to CORSTIME seconds"),
]
hdu0 = fits.PrimaryHDU(header=fits.Header(cards))

In [32]:
hdu0

Now we have the primary header, we can make the first data extension

In [33]:
# Make the image extension
hdu1 = fits.ImageHDU(data=dat, name="SCIENCE")

# These cards tell us about the format
cards = [
    ("TTYPE1", "COUNTS", "data title: raw pixel counts"),
    ("TFORM1", "J       ", "data format: images of unsigned 32-bit integers "),
    ("TUNIT1", "count   ", "data units: count"),
]
# Append the cards to the first extension header
_ = [hdu1.header.append(c) for c in cards]

In [34]:
hdu1

We can make a second exension which is the table of star paramters for the ROIs

In [35]:
hdu2 = fits_from_table(tab)
hdu2.header.append(("EXTNAME", "ROITABLE", "name of extension"))
cards = [
    ("NROI", 9, "number of regions of interest"),
    ("ROISTRTX", 512, "region of interest origin in column"),
    ("ROISTRTY", 512, "region of interest origin in row"),
    ("ROISIZEX", 1024, "region of interest size in column"),
    ("ROISIZEY", 1024, "region of interest size in row"),
]

_ = [hdu2.header.append(c) for c in cards]

In [36]:
hdu2

Now we have a primary extension, the first extension is the set of images, and the second extension is a table. We can now make them into an `HDUList` object and write it to a file

In [37]:
hdulist = fits.HDUList([hdu0, hdu1, hdu2])

In [38]:
hdulist.info()

Filename: (No file associated with this HDUList)
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      23   ()      
  1  SCIENCE       1 ImageHDU        12   (256, 256, 3)   int32   
  2  ROITABLE      1 TableHDU        22   8R x 2C   ['D25.17', 'D25.17']   


In [39]:
hdulist.writeto('test.fits', overwrite=True)

Note there are other extension types you might want to use (e.g. compressed image formats) and you can read about them here on the astropy documentation here https://docs.astropy.org/en/stable/io/fits/

For image extensions, I believe there is a shape where 3D images are compressed best, and I usually put them in as shape (ntime, npixel1, npixel2), but it might be worth playing around with.